<a href="https://www.nlpfromscratch.com?utm_source=notebook&utm_medium=nb-header"><center><img src="https://drive.google.com/uc?export=view&id=1-lt6Uft8lgBG9jPD0dO6w3dAcv_EUQRP"></center></a>

# Large Language Models and Generative Text

Copyright, NLP from scratch, 2023.

[nlpfromscratch.com](https://www.nlpfromscratch.com)

------------

## Introduction 🎬
In this notebook, we will explore Large Language Models (LLMs) for generative text, and show how they can be leveraged the open source libraries from [Hugging Face](https://huggingface.co/).

This notebook is best run in [Google Colab](https://colab.research.google.com/), where the majority of dependencies are already installed. However, if you wish to run the notebook locally, please follow the [directions for setting up a local environment](https://drive.google.com/file/d/1EV1seK-dUHRCzj2EDuu3ETAhUyjzOGRd/view?usp=drive_link) and you may then download the notebook as a `.ipynb` and run in either Jupyter or Jupyterlab.

Since we will be using GPU in this notebook for compute-intensive tasks, please ensure that if running on Colab the runtime type is set to GPU. In the menu in Colab, select *Runtime -> Change runtime type*, then select T4 GPU (if using Colab Free) or another GPU instance type if using Colab Pro.

<center><img src="https://drive.google.com/uc?export=view&id=1t5rRQIHd12xVFXHRvzaDnjR2Vd-QJQPf" width="50%"/></center><br/>

Though Google Colab comes with many useful data science libraries included by default (including Pytorch), the Hugging Face libraries are not, so we will first install those here using `pip`, as they will be used in the remainder of the notebook.

- The `transformers` library, for general usage of transformer models
- The `datasets` library, for working with datasets hosted on Hugging Face
- The `accelerate` library, for using GPU for inference
- The `evaluate` library, for metrics for measuring model performance in training
- The `bitsandbytes` library for model quantization
- The `peft` library, for efficient fine-tuning of models in the second half of the workshop
- The `huggingface_hub` library, for interacting with models on the Hugging Face hub

We will also be using custom datasets from the NLP from scratch [github repo](https://github.com/nlpfromscratch/datasets/) and so we will clone this repo to have these all available locally.



In [ ]:
!git clone https://github.com/nlpfromscratch/datasets.git

Cloning into 'datasets'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 70 (delta 14), reused 61 (delta 8), pack-reused 0
Receiving objects: 100% (70/70), 34.61 MiB | 16.89 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Updating files: 100% (27/27), done.


In [ ]:
!pip install transformers datasets accelerate evaluate bitsandbytes peft huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00


## Large Language Models (LLMs) and the Transformer Architecture

What is a large language language model (LLM)? While there is no universally accepted definition for an LLM, large language models are a type of deep learning model that is understood be both very large in size (number of parameters) and also trained on very large datasets. Lately, the datasets used to train LLMs have grown increasingly in size and it is not unusual for these to represent double digit percentages of the web.

LLMs are a type of *deep learning* model, also known as a neural network, a type of machine learning model that seeks to imitate the structure of the human brain. Traditional deep learning models take inputs - structured data found in rows and columns like in a database, or unstructured data like images, video, audio, or, in the case of natural language processing (NLP) models, free-form text - and use these to make predictions about a target variables associated with each observation (row) in the input data. For example, in the case of a deep learning models for computer vision, the input could be a dataset of images and the output to predict an associated data label (which Japanese character is this?).

<center><img src="https://drive.google.com/uc?export=download&id=1062YuHYxriHVg4km1NdiKtFx3_MEpYXx"/></center>

Deep learning models are composed of *layers*, each of which is composed of *nodes*, and each of the nodes has inputs which come from previous layers and associated *weights* which are multiplied by each of the inputs. Collectively, these weights are referred to as the model *parameters* we mentioned earlier, and what is learned in the "learning" of deep learning is the optimal values for these numeric parameters to best predict the outcome.

Large language models are a category of deep learning models with the following properties:
- They primarily work with language data (*i.e.* text), either as input, or output, or both - either solely or in conjuction with other data types (images, audio, video, etc.), in which case the LLMs are referred to as *multimodal*.
- As mentioned above, LLMs are understood to be both very large in size in both model size (hundreds of millions, billions, hundreds of billions, or even *trillions* of parameters) as well as trained on very large datasets, comprising hundreds of millions or billions of tokens.

It is from the latter of these two properties that the remarkable capabilities of recent LLMs has arisen. There is one another important fundamental development which lead to the rise of LLMs as we know them today and that is the transformer architecture.


### The Transformer Architecture

We refer to the structure of a deep learning network as its *architecture* - the example showed in the previous section is that of the simplest type of deep learning model architecture referred to as a *feed-forward* or *fully connected* neural network, since the outputs of each node in the hidden layers serve as input for each node in the following layer.

There are many different types of more complex and specialized architectures for specific tasks and use cases in deep learning. This have arisen from years of research in academia and application in industry. For example, convolution neural networks (CNNs) are the standard for working with computer vision tasks and have different types of layers specifically suited for this.

The Transformer architecture is an enitrely new type of neural network architecture of which the vast majority (but not all) large language models are. The Transformer was introduced fairly recently in the paper [Attention is All You Need](https://arxiv.org/abs/1706.03762) (Vaswani et al, 2018) by researchers from Google Brain. As an interesting side note, contributors to the paper have moved on to notable AI startups - Aiden Gomez went on to be one of the founders of the OpenAI competitior [Cohere](https://en.wikipedia.org/wiki/Cohere) and Noam Shazeer to co-found [Character.ai](https://en.wikipedia.org/wiki/Character.ai). While the transformer architecture was originally devised to be applied to a text-to-text translation task (English and German) and work with what now comparatively is a very small dataset, it has now been found to be broadly applicable and highly performant to a very wide variety of natural language processing tasks and represents the state-of-the-art for most of these, ranging from text generation which will cover in this workshop, to other applications such as image generation, such as with OpenAI's [DALL-E](https://en.wikipedia.org/wiki/DALL-E) models.

<center><img src="https://drive.google.com/uc?export=download&id=106TIrXE8PP7BKni3MCwxidzkgA_dQrR8"/ width="50%">
<br/>
<caption> The Transformer Architecture, as originally presented in the Attention is All You Need paper </caption></center>

While the mathematical and technical details of the transformer are very complex, we will not dive too deeply into these here - there are many resources out there available which cover this - for example, [Transformers from Scratch](https://peterbloem.nl/blog/transformers) and the reimplementation of the GPT model, [MinGPT](https://github.com/karpathy/minGPT) by Andrej Karpathy, previously at Tesla and now back at OpenAI.

At a high level, a couple important points to note before we dive into working with LLMs in code.


### Encoder and Decoder Models

The transformer model is made up of two large blocks: the *encoder* block on the left, and the *decoder* block on the right. While the original transformer architecture was comprised of both of these, there are now specific models which are composed of stacked blocks of each type. For example, models in the research can be *encoder-only* or *decoder-only* models, or a fully transformer with both encoder and decoder. It should be noted that, in practice, these would all still be referred to as transformer models even though the former two are not "full" transformers based upon the strict definition of the architecture.

Encoder and decoder models have different tasks to which they are well-suited. Generally speaking, encoder models take text as input and produce a higher dimensional representation of the dataset (corpus) of text - an *embedding* - on which they are trained. You will also sometime hear encoder models referred to *autoencoding* models, as they perform a similar task to the traditional [autoencoder model](https://en.wikipedia.org/wiki/Autoencoder) in deep learning.

<br/>
<center><img src="../assets/types_of_transformers.png" width="75%"/>
<br/>
<caption> Types of Transformers. Image Credit: <a href="https://www.comet.com/site/blog/explainable-ai-for-transformers/">Abby Morgan</a></caption>
</center>

Decoder models, on the other hand, take inputs and produces output probabilities. Most commonly, these are known for doing text generation, where the model take a sequence of text as input and makes predictions about the most likely occuring words which come next, as made famous by the [Generative Pretrained Transformer (GPT)](https://en.wikipedia.org/wiki/Generative_pre-trained_transformer) model by OpenAI which we will work with shortly. You will also hear decoder-only models referred to as *autoregressive* models, as they take their own outputs as inputs (in order to make predictions about a sequence of text, word-by-word) and uses these to predict probabilities for the next word (a regression task). As we will see shortly, this type of task in natural language processing is also referred to as *causal language modeling*.

## Working with Generative Text Models

### Use Cases for Generative Text Models



**Code autocompletion and AI-assisted coding**:

<img src="https://drive.google.com/uc?export=download&id=1F202oOM4LHK_bma0A8y_wGO4W-GnKRLF" width="33%"/>

Microsoft’s [Github Copilot](https://github.com/features/copilot) was launched in June 2022. Initially, more that ¼ of developers’ code files on average were generated by GitHub Copilot, and today with widespread adoption this is close to nearly half (~46%) and has been used by over 1M developers. In October 2023, Copilot [surpassed $100M](https://twitter.com/swyx/status/1711792178031460618) in annually recurring revenue.

**Writing Assistants for creativity and copywriting**:

<img src="https://drive.google.com/uc?export=download&id=1F9Bm0KUt9GqH1jNM7nlNJsCLRgcYkrVK" width="33%"/>

AI writing assistants have arisen for improved productivity and content creation for marketing, sales, creative, and numerous other areas. For example, Google has made this a part of their core offerings with their announcement of [Duet AI](https://workspace.google.com/solutions/ai/) and Canva has introduced [MagicWrite](https://www.canva.com/magic-write/) based upon OpenAI’s offerings.

**Entertainment and Social Uses:**

<img src="https://drive.google.com/uc?export=download&id=1FFxNDJ6X8U3A-r3oT_Sl2aUd34br8Y11" width="30%"/>

Training generative language models on specific datasets has allowed to give them “personality”. [Character.ai](http://Character.ai) was created by developers who previously worked on Google’s LaMDA model, offers chatbots based upon fictional characters and famous individuals. It is #2 on Anderssen- Horowitz’s list of [top 50 most popular GenAI web products](https://a16z.com/how-are-consumers-using-generative-ai/) (Sept 2023).

### Loading our first Hugging Face Model

In this section, we will start generating text with our first large language model, [GPT-2](https://huggingface.co/gpt2) and explore some of the parameters which affect the outputs from a generative text model.

The GPT-2 (Generative Pre-trained Transformer 2) model was the last of the series of GPT models from OpenAI which was "open". Following its release in 2019, GPT-3 and subsequent models did not have their weights made available publicly (and in the case for more recent models such as GPT-4, nor the details of their training data and training process).

We can easily work with GPT-2 in [Hugging Face](https://www.huggingface.co). The easiest way to get results as quickly as possible is to use a [pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines) to generate text *i.e.* to perform inference.

First, we import the Pipeline class from the `transformers` library, then creator an instance of it, specifying the model type we wish to use. In this case, we want to use GPT-2, which is hosted Hugging Face themselves, not as part of a user repo, so the URL for it is just `gpt2`.

Pipelines can also be for a large variety of different tasks, we must specify that the pipeline is for text generation.

Finally, we check whether GPU is available (it should be on Colab) and if so, set the model to use GPU. This requires importing [pytorch](https://en.wikipedia.org/wiki/PyTorch) (`torch`), which is the first line of code.

In [ ]:
import torch
from transformers import pipeline

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create a pipeline of the GPT-2 model
gpt2_pipeline = pipeline('text-generation', model='gpt2', device=device)

# Create 3 output generations
outputs = gpt2_pipeline("I love applesauce!", max_length=40, num_return_sequences=3)

# Display the first output
print(outputs)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I love applesauce!'}, {'generated_text': 'I love applesauce! My girlfriend even brought us all home," she said.\n\nShe didn\'t mention the recent spate of death threats that have been lobbed at her and many of her'}, {'generated_text': "I love applesauce! I feel great every time. I love the apple flavor but I just can't take it anymore. I've tried two other ways, maybe I'll just get it right"}]


We can see that even though we've only a few lines of code, Hugging Face has pulled down over half a gigabyte of data! These are the [model weights for GPT-2](https://huggingface.co/gpt2/blob/main/pytorch_model.bin). For this part of the notebook, we are also using a smaller version of GPT - the full GPT-2 model, [GPT2-XL](https://huggingface.co/gpt2-xl) is ~6.5 GB!

Let's take a look at what's in the pipeline - it will contain both a `tokenizer`, for breaking inputs up into the tokens that GPT-2 expects, as well as a `model`, in this case, our GPT-2 model:

In [ ]:
# Check the class of the tokenizer in the pipeline
type(gpt2_pipeline.tokenizer)

transformers.models.gpt2.tokenization_gpt2_fast.GPT2TokenizerFast

In [ ]:
# Check the class of the model in the pipeline
type(gpt2_pipeline.model)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

Furthermore, we can check the number of parameters of any Hugging Face model by calling the `num_parameters` method of a model object. How many parameters (weights) does our GPT-2 model have?

In [ ]:
# Get the number of model parameters, format nicely with an f-string
f"{gpt2_pipeline.model.num_parameters():,}"

'124,439,808'

Here we can see our GPT-2 model has just over 124 million parameters. Now we can move forward into generating some text using the model.

### Generating Text
In this section, we will generate some text using the GPT-2 model, and also explore the different decoding methods for doing so, and the effect they have on outputs.

First, let us generator text from the pipeline using the default behavior. To do this, we simply pass in a string of text and no other arguments:

In [ ]:
my_input_string = "The rain in Spain falls mainly in the plain"

# Generate output
output = gpt2_pipeline(my_input_string)

# Display
print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The rain in Spain falls mainly in the plain, so it comes on top of us like an ointment to help us to put our nose up.\n\nWith the wet, there is a bit of moisture in them. The rainfall in Spain'}]


We can see that the model has actually generated a `list` of outputs, each which are a dictionary. Let's take a look at the first output:

In [ ]:
output[0]

{'generated_text': 'The rain in Spain falls mainly in the plain, so it comes on top of us like an ointment to help us to put our nose up.\n\nWith the wet, there is a bit of moisture in them. The rainfall in Spain'}

This is just a dictionary with a single key, `generated_text`, which contains both the input we sent into the model, as well as the tokens the model predicted. We can display the output a little more nicely using the [Markdown](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html#IPython.display.Markdown) object from IPython (Jupyter), to render it inline like the rest of the text in our notebook here.

In [ ]:
from IPython.display import Markdown

display(Markdown("---")) # dividing line
display(Markdown(output[0]['generated_text']))
display(Markdown("---")) # dividing line

---

The rain in Spain falls mainly in the plain, so it comes on top of us like an ointment to help us to put our nose up.

With the wet, there is a bit of moisture in them. The rainfall in Spain

---

There, that's better! Now the text is displayed nicely with dividers. Let's move on now to different parameters we have at our disposal for how a model generates text, or as in the language of LLMs, different *decoding strategies*.

### Text Decoding Strategies

As we will see in this section, there is some complexity to creating text outputs with generative language models. Creating new outputs from a given prompt is not as simple as entering the input and getting a predicted output. Generative text models have parameters which control the amount of variability in their outputs; this is a desirable quality to make the outputs seem both more realistic (as if from a human) and  variety being injected into the model outputs also increases the likelihood of reaching a novel result that is pleasing to the user and deemed to be "good".

First, we will consider the simplest (vanilla) text generation approaches in order to both gradually work our way up, and also contrast with, using them with methods which introduce variety and "creativity". The two simplest decoding methods for text generation we will consider first are *greedy search* and *beam search*.


#### Greedy Search

Greedy search is the simplest text generation approach: in this case, no variety is introduced as all. Recall a text generation model takes a sequence of input tokens and its task is to predict the next token given the input. For greedy search, the next predicted token is always just that with the highest probability.

<center>
<img src="https://drive.google.com/uc?export=download&id=182wa0OBECLTS218FjA3EZIu3iiQB1jX2" width="75%"/>
</center>
<caption><i> Greedy Search. Here, for the next two tokens the words "plain" and "which" are selected, as they have the highest individual probabilities. </i></caption>

Mathematically speaking, given an input sequence of tokens $x_1, x_2, x_3...$, the model seeks to produce an output $y_t$ at step $t$. Since generative text models (decoder models) are *autoregressive* and make predictions based upon previous predictions after the initial input, mathematically we can express the prediction task as:

$ P(y_t|y_1, y_2, ..., y_{t-1},x)$

Greedy search just takes the highest probability token for each prediction. Thus for the vocabulary and different calculated probabilities by the model, this is expressed mathmeatically as:

$y_t = argmax_{y \in V}P(y|y_1,y_2,...,y_{t-1},x)$

Let's take a look at this with GPT-2, to do this we will play around with the [parameters](https://huggingface.co/docs/transformers/generation_strategies#customize-text-generation) we can pass to the call to `.generate` on our model in Hugging Face.

Now that we are stepping outside of the pipeline abstraction and working in more detail, we should probably initialize a tokenizer and model, and work with these separately, passing the outputs of the tokenizer to the model directly. To do this, we will be leveraging some of the [Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto) in Hugging Face.

Since we are doing text generation, *i.e.* [causal language modeling](https://huggingface.co/docs/transformers/tasks/language_modeling), we will using the `AutoModelforCausalLM` class to create the GPT-2 model, as well as creating a tokenizer using `AutoTokenizer`.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(device)

# Text input string
input_string = "The rain in Spain falls mainly in the plain"

Great, now we have the tokenizer, model, and input string. We pass the input string into the tokenizer to get a back a list of token ids, as well as the attention mask for the transformer:

In [ ]:
# encode context the generation is conditioned on
model_inputs = tokenizer(input_string, return_tensors='pt').to(device)

In [ ]:
# What is the result?
print(model_inputs)

{'input_ids': tensor([[ 464, 6290,  287, 8602, 8953, 8384,  287,  262, 8631]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


We then pass this to the model method `generate`. Here we use the "double-star" syntax, where the dictionary that is passed in is "unpacked" by python, so the function receives separate arguments for `input_ids` and `attention_mask` from the associated values. Let's take a look at the result:

In [ ]:
# Do greedy generation to generate the output token ids
greedy_output = model.generate(**model_inputs)

print(greedy_output)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[  464,  6290,   287,  8602,  8953,  8384,   287,   262,  8631,   286,
           262, 50206, 12010,    11,   475,   340,   318,   635,   287,   262]],
       device='cuda:0')


We can see that the result is just a list of integers. These are the token ids that were predicted by the model as the next most likely, based upon the tokenizer vocabulary. So we can convert these token ids back into text by passing them through the tokenizer as a final step:

In [ ]:
# Decode the tokens back to text using the tokenizer
output_string = tokenizer.decode(greedy_output[0])

# Print the result
display(Markdown("---")) # dividing line
display(Markdown(output_string))
display(Markdown("---")) # dividing line

---

The rain in Spain falls mainly in the plain of the Canary Islands, but it is also in the

---

And that's it! The whole text generation process goes like this:
1. Instantiate tokenizer and model
2. Pass input string to tokenizer to generate token ids and attention mask
3. Generate output token ids (predictions) from the model
4. Decode output token ids back into text using tokenizer

We can visualize the whole process with the figure below:

<center>
<img src="https://drive.google.com/uc?export=download&id=1-mYXn0obOPCjWg0wGHjEYlZL9b7wgUXh" width="75%"/>
</center>

It should be noted that with greedy search, we will always be picking the most likely output tokens, and so the final result will be completely determinstic and the same each time. We can see this with the behavior of the model below by generating the same output over and over:

In [ ]:
# Initial generation
greedy_output = model.generate(**model_inputs)
output_string = tokenizer.decode(greedy_output[0])

In [ ]:
# Output
print(output_string)

The rain in Spain falls mainly in the plain of the Canary Islands, but it is also in the


In [ ]:
# Second generation
greedy_output2 = model.generate(**model_inputs)
output_string2 = tokenizer.decode(greedy_output2[0])

In [ ]:
# Output
print(output_string2)

The rain in Spain falls mainly in the plain of the Canary Islands, but it is also in the


We can see that we will always get the same result as an output here based on the model. Let us now explore other approaches for generating text which generate the series of output tokens based on different approaches.

#### Beam Search

Beam search is an improvement on greedy search which considers the most likely sequence of tokens *together*, based on their respective probabilities, as opposed to just taking the most probable individual token at each timestep.

A *beam width* is specified, and over the width of the beam (number of generated tokens), the combination of tokens with the highest collective probability is selected, as opposed to just selecting the individual token with the highest probability, as with greedy search.

<center>
<img src="https://drive.google.com/uc?export=download&id=1875NvbUveU2ya3IdMpQfKfa8k5D9h5Tu" width="75%"/>
</center>
<caption><i> Beam Search. Here, for the next two tokens the words "meadow" and "grasses" are selected, as they joint probability of 0.36 (0.4 x 0.9) is greater than that of the tokens selected in greedy search which is 0.33 (0.6 x 0.55). </i></caption>

A couple points to note about beam search is that searching over a larger sequence of tokens (*i.e.* increasing `beam_size`) will result in significantly improved quality of outputs at the cost of increased computation.

There is a "law of diminishing returns" with beam search: typically there is a saturation point beyond which increasing the beam size does not significantly change the most likely generated sequence, as the probabilities are dominated by the product of the most frequently occurring tokens in the sequence considered by beam search.

Generally speaking, beam search can lead to repetitive outputs for open-ended generation. This is why it and greedy search are used in conjuction with sampling.

To generate text with beam search in Hugging Face, we set the `num_beams` parameter to a value greater than 1 (which would be equivalent to greedy search) and `early_stopping=True`, so generation finishes when all beams pass back an "end of string" (EOS) token.

We have already created our tokenizer and model, so this can just be done in the call to `model.generate()`:

In [ ]:
# Text input string
input_string = "The rain in Spain falls mainly in the plain"

# Model input
model_inputs = tokenizer(input_string, return_tensors='pt').to(device)

# Generate output with beam search
greedy_output = model.generate(**model_inputs, num_beams=10, early_stopping=True)

# Decode the output
output_string = tokenizer.decode(greedy_output[0])

In [ ]:
display(Markdown(output_string))

The rain in Spain falls mainly in the plain to the south of the city of Barcelona.



We can see that beam search has returned quite a different result from that of greedy search, by looking over the collective probabilities of a number of predicted token possibilities, instead of just each following token.

#### Sampling Strategies

While the different search decoding strategies provide some varaibility in the outputs of a generative text model, they are still determinisitc in their outputs, and this can lead to either a.) poor outputs or b.) repeated identical outputs, the latter of which is not a desireable traits for end users.

As such, there also exist different *sampling strategies* for introducing variability and novelty into the outputs of generative text models. The three main parameters available for different sampling strategies are *temperature*, *top-p,* and *top-k*.


##### Temperature

The temperature is a factor which normalizes or "smooths out" the output probabilities of predicted tokens. In practice, it is used to control the variability (or randomness, or "creativity") of the outputs of a model.

Mathematically speaking, calculating the model probability for predicting any individual token as the next one, such that all probabilities lie between zero and one and sum to one, is attained using the softmax function:

$ P(y_i) = \frac{e^{z_i}}{\sum_{j=1}^{N}e^{z_j}} $

where:
- $P(y_i)$ is the probability of selecting the $i$th token.
- $z_i$ is the logit, the raw score or output, from the model for token $i$
- and $N$ is the total size of the vocabulary

we introduce a new variable $\tau$ for temperature and update the probability formula as below:

$P(y_i) = \frac{e^{z_i / \tau}}{\sum_{j=1}^{N}e^{z_j / \tau}}$

Given the above, if $\tau = 1$, the formula for the probabilities, and thus the behavior of the model, is unchanged. It can be shown that as $\tau \to \infty$, that $P(y_i) \to 1$ for all $i$, and so the likelihood of any token predicted becomes equal. This results in a completely uniform distribution of probabilities acrosss all possible tokens.

On the other hand, as $\tau \to 0$, the probability for any given token can be represented by:

$$
P(i)=\begin{cases}
    1 & \text{if $i$ is max probability}\\
    0 & \text{otherwise}
  \end{cases}
$$

That is to say, the most likely token will have a probability of 1, and then others will have their probabilities set to 0, and the output of the model will be completely deterministic.

To put in another way, setting a low value to temperature (value of 0) means that the most likely next tokens will always be returned, whereas setting higher values to temperature flattens the probabilities across the different possible tokens, resulting in increasingly random outputs for greater values of $\tau$.

This is visualized in the figure below:

<center>
<img src="https://drive.google.com/uc?export=download&id=1EEM8re3w97gFONaeRf6Sp76TVmtzoZaC" width="80%"/>
</center>
<center><caption> Visualizing the effect of changing temperature on next token probabilities </caption></center>

There is a balance to be struck, as too low a temperature will result in a model always returning the same output for a given input - that is, acting deterministically - whereas setting the temperature too high can result in garbled and incoherent.

Now let's try experimenting with changing the temperature parameter for text geenration using GPT-2. In Hugging Face, this is controlled by the `temperature` parameter in either calls to a model pipeline, in directly in the text generation call in `model.generate()`. We must also set the `do_sample=True` argument, to tell Hugging Face to use sampling and not to do greedy search.

First, let's set a temperature (close to that) of 0, which will always result in the most likely token be chosen. Note that this is equivalent to greedy search:

In [ ]:
# Text input string
input_string = "The rain in Spain falls mainly in the plain"

# Generation =  temperature ~= 0 - deterministic
model_inputs = tokenizer(input_string, return_tensors='pt').to(device)
zero_temp_output = model.generate(**model_inputs, temperature=0.00001, do_sample=True, num_return_sequences=3)

# Iterate over outputs and display in markdown
display(Markdown("---"))

for output in zero_temp_output:
  output_string = tokenizer.decode(output)
  display(Markdown(output_string))

display(Markdown("---"))

---

The rain in Spain falls mainly in the plain of the Canary Islands, but it is also in the

The rain in Spain falls mainly in the plain of the Canary Islands, but it is also in the

The rain in Spain falls mainly in the plain of the Canary Islands, but it is also in the

---

We see that the same output is returned as before, and we can run the above cell multiple times and always get back the same input. Now let's set the temperature to 1, which will leave the next token probabilities unchanged. In this case, we should be able to get different outputs:

In [ ]:
# Text input string
input_string = "The rain in Spain falls mainly in the plain"

# Generation: temperature = 1, default behavior
model_inputs = tokenizer(input_string, return_tensors='pt').to(device)
temp1_output = model.generate(**model_inputs, temperature=1, do_sample=True, num_return_sequences=3)

# Iterate over outputs and display in markdown
display(Markdown("---"))

for output in temp1_output:
  output_string = tokenizer.decode(output)
  display(Markdown(output_string))

display(Markdown("---"))

---

The rain in Spain falls mainly in the plain of Barcelona city centre in the north. It rains on

The rain in Spain falls mainly in the plain of Sólúm where the rivers end in

The rain in Spain falls mainly in the plain of Iberian Peninsula, to the south of the

---

Cool, those all seem like reasonable outputs, even though they are all different. We have introduced some variability into the model outputs which makes for novelty.

Finally, let's really crank up the temperature! This will make all output tokens equally likely, resulting in very "creative" outputs:

In [ ]:
# Text input string
input_string = "The rain in Spain falls mainly in the plain"

# Generation: temperature = 1B, all tokens equally likely
model_inputs = tokenizer(input_string, return_tensors='pt').to(device)
high_temp_output = model.generate(**model_inputs, temperature=1.0e9, do_sample=True, num_return_sequences=3)

# Iterate over outputs and display in markdown
display(Markdown("---"))

for output in high_temp_output:
  output_string = tokenizer.decode(output)
  display(Markdown(output_string))

display(Markdown("---"))

---

The rain in Spain falls mainly in the plainlands as it drihes back in off France, or

The rain in Spain falls mainly in the plain regions like Ligue D (near Basir da M

The rain in Spain falls mainly in the plain, just behind what seemed less developed communities like Castil

---

As we can see above, setting a high value for temperature results in more "creative" outputs but some of these are less coherent than those with lower temperature.

Now let us consider further sampling strategies for introducing variability in model outputs whilst attempting to maintain the quality thereof.

##### Top-p & Top-k sampling

Unlike temperature, which changes the different calculated probabilities of the next token, *top-p* and *top-k* instead function by reducing the size of the set of possible tokens to choose from. Though are differently in how they are applied, they both restrict the set of possible next tokens to only the most likely ones above a specified threshold, and then redistribute the probability mass amongst this smaller set. They are typically used in conjunction with temperature to produce varied but still comprehensible outputs.

In *top-k* sampling, instead of calculating probabilities and sampling from all possible tokens, a cutoff integer value $k$ is specified, and only the top $k$ ranked tokens are used as the set of possible next tokens. The total probability (summing to 1) is redistributed amongst these top $k$ tokens.

This is illustated in the figure below. Instead of choosing from all possible next words, only the top 5 words would be considered, and the probabilities would be redistributed amongst them:

<center>
<img src="https://drive.google.com/uc?export=download&id=1FGD7yGPCBrHixsUyMX2U7nQZRMCbAD26" width="50%"/></center>
<center><caption> Top-k sampling: only the most probable tokens above and including rank $k$ are kept </caption></center>

*Top-p*, or *nucleus sampling* differs in that instead of specifying a rank $k$ and taking the most probable tokens this rank or above, in top-p a probability threshold $p$ is specified, and only the top tokens which a combined probability above this threshold are kept in the set of next possible tokens. This differs from top-k in that we don't specify the size of the set of next tokens, only the total probability.

Coming back to our previous example, here using top-p, we wish only to keep tokens which have a combined probability equal to or above a threshold 0.8. In this case the top four most likely next tokens meet this criteria (as $0.5 + 0.15 + 0.1 + 0.05 = 0.8$) so the total probabilty would be redistributed only amongst them:

<center>
<img src="https://drive.google.com/uc?export=download&id=1FHlQL-kkIK_7-kO4sM-tmIyZxn8jdkLh" width="50%"/></center>
<center><caption> Top-p sampling: only the tokens with cumulative probability above the specified threshold $p$ are kept </caption></center>

In Hugging Face, top-k and top-p sampling can be used by specifying them in with the arguments `top_k` and `top_p` respectively. `top_k` is an integer value, and `top_p` a floating point between 0 and 1.

Note that both of these will still just return the most likely sequences (deterministically) and so should be combined with beam search and/or temperature. These allows returning multiple outputs with `num_return_sequences` as we've seen before:

In [ ]:
# Text input string
input_string = "The rain in Spain falls mainly in the plain"
model_inputs = tokenizer(input_string, return_tensors='pt').to(device)

# Generation - Top-k & Top-p
top_k_output = model.generate(**model_inputs, top_k=30, do_sample=True, num_return_sequences=3)
top_p_output = model.generate(**model_inputs, top_p=0.5, do_sample=True, num_return_sequences=3)

# Top K
display(Markdown("---"))
display(Markdown("Top-k, $k=30$:"))
for output in top_k_output:
  output_string = tokenizer.decode(output)
  display(Markdown(output_string))

# Low Top K
display(Markdown("---"))
display(Markdown("Top p, $p=0.5$:"))
for output in top_p_output:
  output_string = tokenizer.decode(output)
  display(Markdown(output_string))
display(Markdown("---"))

---

Top-k, $k=30$:

The rain in Spain falls mainly in the plain of Catalonia, an important city which lies along the coast

The rain in Spain falls mainly in the plain of Sertra on a Saturday, but this week

The rain in Spain falls mainly in the plain which is part of the Caja Rural (Cran

...

Top p, $p=0.5$:

The rain in Spain falls mainly in the plain, and there are no visible signs of rain in the

The rain in Spain falls mainly in the plain of the Canary Islands, which is the home of the

The rain in Spain falls mainly in the plain of Madrid, but in the mountains of the Andalus

---

Top-p and Top-K can be used in conjunction, to avoid very low ranked words while allowing for variability. In pratice, this requires a fair bit of trial and error to find good values for $k$ and/or $p$, combined with temperature.

In [ ]:
# Putting it all together
outputs = model.generate(
    **model_inputs,
    do_sample=True,
    top_k=30,
    top_p=0.5,
    temperature=1.5,
    num_return_sequences=3,
)

display(Markdown("---"))
for output in outputs:
  output_string = tokenizer.decode(output)
  display(Markdown(output_string))
display(Markdown("---"))

---

The rain in Spain falls mainly in the plain, which is covered with a thick layer of thick layer

The rain in Spain falls mainly in the plain of the Andalusia Mountains, which have been a

The rain in Spain falls mainly in the plain. The city's roads are covered with asphalt, and

---

Great! Now let's move onto the next section of the notebook on fine-tuning models. We should restart the kernel of the notebook here to clear the RAM, then re-import the modules we need:

In [ ]:
import torch
from IPython.display import Markdown
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

## Fine-tuning Large Language Models

Now that we have covered some of the fundamentals of working with pre-trained large language models for generative text, we will progress to the more advanced task of adapting pre-trained models to specific tasks or datasets to change their behavior.

Adapting a pre-trained LLM for a new problem or specific dataset by updating its parameters through further training is referred to as [fine-tuning](https://en.wikipedia.org/wiki/Fine-tuning_(deep_learning)). Though the nomenclature has evolved and the terms are now sometimes used interchangeably, this technique is a type of [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning). Other methods for transfer learning do exist, and so the term "fine-tuning" should be more specifically use to refer to cases in which model weights are updated (or new model weights are added and optimized) against a new dataset or objective.

In practice, the term *fine-tuning* is more commonly now used with respect to LLMs, while that of *transfer learning* to refer to the larger group of approaches, usually inside the domain of machine learning and deep learning outside of language models.

One of the most common applications for fine-tuning is to take a well-performing pre-trained language model (foundation model) as a base and adapt this to a new classification task. This is commonly done with the [BERT model](https://en.wikipedia.org/wiki/BERT_(language_model) as a base, which has already learned powerful and meaningful representations of language, and adapt it to other NLP tasks, for example, by attaching a classification "head", or even by having another simple classifier model take the outputs of the BERT model as inputs for a classification task.

<center>
<img src="https://drive.google.com/uc?export=download&id=1FI4_Pq_MVTdjMDpMIB76qnbaBZ63AoEF" width="75%"/>
</center>
<center><caption> Adapting BERT to a new text NLP task via fine-tuning </caption></center>

There is an example of the above, for full fine-tuning of BERT (no frozen layers) in the [Hugging Face documentation](https://huggingface.co/docs/transformers/training#fine-tune-a-pretrained-model).

In the remainder of this workshop, we will focus on fully fine-tuning the GPT-2 model we've been working with already as a "hello world" example, and see if we can change the behavior (outputs) of this generative text model with fine-tuning.

### Fine-tuning a model using a Hugging Face script

By far the easiest way to fine-tune a model with little coding required is to use a pre-written script for this purpose. Hugging Face has a provided scripts for fine-tuning models for masked language modeling (MLM) and causal language modeling (CLM), *i.e.* text generation in the [github repo for Transformers](https://github.com/huggingface/transformers/tree/main/examples/pytorch). This repo contains a large number of scripts for all kinds of applications of the `transformers` library and is a great resource. There is also a wide selection of [Jupyter notebooks](https://github.com/huggingface/transformers/tree/main/notebooks) illustrating these as well.

The script for fine-tuning a model for text generation is [run_clm.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_clm.py).

In the workshop we will focus on writing and understanding the code to fine-tune models, however the invocation for doing so using the script is provided below, and we show a simple example of training the [distilgpt2](https://huggingface.co/distilgpt2) model quickly on a toy dataset, the [`tinyshakespeare`](https://github.com/nlpfromscratch/datasets/tree/master/tinyshakespeare) dataset from Andrej Karpathy.

In [ ]:
# Reset the encoding (this is a bug from Hugging Face / Colab)
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# Download the script
!wget https://github.com/huggingface/transformers/raw/main/examples/tensorflow/language-modeling/run_clm.py

--2023-11-20 15:09:54--  https://github.com/huggingface/transformers/raw/main/examples/tensorflow/language-modeling/run_clm.py
Resolving github.com (github.com)... 20.29.134.23
Connecting to github.com (github.com)|20.29.134.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/huggingface/transformers/main/examples/tensorflow/language-modeling/run_clm.py [following]
--2023-11-20 15:09:54--  https://raw.githubusercontent.com/huggingface/transformers/main/examples/tensorflow/language-modeling/run_clm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29630 (29K) [text/plain]
Saving to: ‘run_clm.py’

run_clm.py          100%[===================>]  28.94K  --.-KB/s    in 0.001s  

2023-11-20 15:09:54

Let's take a look at the data:

In [ ]:
!ls datasets/tinyshakespeare

input.txt  README.md


In [ ]:
f = open('datasets/tinyshakespeare/input.txt','r')

for i in range(0, 5):
  print(f.readline())

First Citizen:

Before we proceed any further, hear me speak.



All:

Speak, speak.



In [ ]:
!python run_clm.py \
    --model_name_or_path "gpt2" \
    --output_dir "gpt2_shakespeare" \
    --train_file "datasets/tinyshakespeare/input.txt" \
    --validation_file "datasets/tinyshakespeare/input.txt" \
    --do_train \
    --do_eval \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --num_train_epochs 1

2023-11-20 15:25:00.805989: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 15:25:00.806061: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 15:25:00.806108: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 15:25:03.483806: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/11c5a3d5811f50298f278a704980280950aedb10/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "g

In [ ]:
# Input
input_text = "ROMEO:"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Load the model into a pipeline, here the argument to is the local path, not the model name on Hugging Face hub
generator = pipeline('text-generation',model='./gpt2_shakespeare', tokenizer=tokenizer, max_length=50)

# Generate the outputs
outputs = generator(input_text, num_return_sequences=3, do_sample=True, temperature=0.8, top_k=5, top_p=0.95, no_repeat_ngram_size=5)

# Print
display(Markdown("---"))
for output in outputs:
  display(Markdown(output['generated_text']))
  display(Markdown("---"))

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt2_shakespeare.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---

ROMEO:

I am not the one of you that will not be the one.

MARTIN:
I am not that.

MADEIN:
I do.

MARY:
What's the

---

ROMEO:
I will make my way to your palace,
I will be with you in your presence,
I will take you with me to my palace
And I will be your guest.

JOSEPH:
I

---

ROMEO:

I am the king of the earth.

KING OF KING OF KING OF THE KING OF THE KING

KING OF THE KING OF KING OFTHE KING OF THE KING of THE KING OF THE

KING OFTHE

---

### Fine-tuning a model in Hugging Face using the Trainer API

In this section, we will write our own code to train a model using the Hugging Face library directly. Essentially, HF acts as a higher-level API around pytorch, handling all of the nitty-gritty lower level details of training for us.

Here, we will work with the [Yoda dataset](https://github.com/nlpfromscratch/datasets/tree/master/yoda) to teach a GPT to speak like our favourite Jedi master. In this case, however, we will work directly with Hugging Face's `Trainer` class to fine-tune our model.

<center>
<img src="https://drive.google.com/uc?export=download&id=1Gz5L_JNPApconeL-9IgBciEseRRKs6t-" width="150px"/>
</center>

Now, we can load our base model that we wish to fine-tune. Doing this type of work is computationally demanding, so again, please make sure you are using a GPU runtime if you are running this notebook in Colab, or have sufficient computing resources (*i.e.* a GPU) if you are choosing to run the notebook locally.

#### Loading the Data

As with all machine learning, we first need data. The Yoda dataset is conveniently stored in the NLP from scratch [datasets repo](https://github.com/nlpfromscratch/datasets) on Github, which we already pulled down at the beginning of the notebook using `git`.

Let's take a look at the data:

In [ ]:
# Take a look at the yoda data
!ls datasets/yoda/

README.md  yoda.csv


We can see there is one data file, `yoda.csv`. This contains all the lines spoken by Yoda in the Star Wars films. Let's take a look at some of the data we'll be using for fine-tuning a generative text model:

In [ ]:
import pandas as pd

# Read
yoda_df = pd.read_csv('datasets/yoda/yoda.csv')

# Show
yoda_df.head()

text
0  The very Republic is threatened, if involved t...
1  Hard to see, the dark side is. Discover who th...
2  With this Naboo queen you must stay, Qui-Gon. ...
3                         May the Force be with you.
4      (Cont'd) Master Qui-Gon more to say have you?

In [ ]:
yoda_df.shape

(103, 1)

In total there are 103 lines of dialog we'll be using for fine-tuning our model. As a general rule, pre-training LLMs requires very large datasets and is highly computationally expensive, whereas fine-tuning them requires much smaller datasets and is not.

Now we will import the `load_dataset` function from the `datasets` library, to [load the CSV file](https://huggingface.co/docs/datasets/loading) into a format the Hugging Face expects. We pass a dictionary with a single key, `train` and value of the filename, here `yoda.csv`. We also pass a string of the `dataset_name`, which is the path (directory) where the data reside:

In [ ]:
from datasets import load_dataset

data_files = {"train": "yoda.csv"}
dataset_name = 'datasets/yoda/'
dataset = load_dataset(dataset_name, data_files=data_files)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Great, that seems to have worked. Let's do a quick check here and take a look at the dataset object:

In [ ]:
# Quick check
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 103
    })
})

In [ ]:
type(dataset)

datasets.dataset_dict.DatasetDict

We can see the dataset is a dataset dictionary from the `datasets` library, and contains a single feature, the column `text`. This is suitable for causal langauge modeling, so we may proceed.

#### Loading the Tokenizer and Model

Now that we have the raw data, we need to preprocess it using a tokenizer. Here, we will just be using an [AutoTokenizer](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoTokenizer) and load the that for the base type of model we are using, which in this case is GPT-2.

We also need to load the base model with the original weights, the model that we will be fine-tuning. Since we are doing causal language modeling (*i.e.* text generation), here we will load GPT-2 using the [AutoModelforCausalLM](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForCausalLM) class.

In [ ]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

# Use GPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Instantiate model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

Let's just do a quick check of our tokenizer and model now. First for tokenizing input text:

In [ ]:
# Generate inputs for model
input = tokenizer("The rain in Spain", return_tensors="pt").to(device)
print(input)

{'input_ids': tensor([[ 464, 6290,  287, 8602]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1]], device='cuda:0')}


Next, we generate the model outputs by passing the input through the model:

In [ ]:
# Generate model outputs
output = model.generate(**input, max_new_tokens=20)
print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[  464,  6290,   287,  8602,   468,   587,   523,  2089,   326,   262,
          1748,   286, 15142,   468,   587,  4137,   284,  1969,   663,  8215,
            13,   198,   198,   464]], device='cuda:0')


Finally, we decode the model's output token ids back to text:

In [ ]:
print(tokenizer.decode(output[0]))

The rain in Spain has been so bad that the city of Barcelona has been forced to close its doors.

The


Great! Everything appears to be working fine. Currently, our input dataset is all freeform text:

In [ ]:
dataset['train'][0:5]

{'text': ['The very Republic is threatened, if involved the Sith are.',
  'Hard to see, the dark side is. Discover who this assassin is, we must.',
  'With this Naboo queen you must stay, Qui-Gon. Protect her.',
  'May the Force be with you.',
  "(Cont'd) Master Qui-Gon more to say have you?"]}

We'll need to apply the tokenizer to each record to create a tokenized version to pass into the model as input. This is the name as what we did above, only now we need to do this for each row in the dataset. To do this, we'll create a simple function then apply it over the entire dataset using the `.map` method:

In [ ]:
# Add a padding token to the tokenizer (required)
tokenizer.pad_token = tokenizer.eos_token

# Define tokenization function using the already instantiated tokenizer
def tokenize_function(data):
    my_tokenizer = tokenizer(data["text"], padding="max_length", truncation=True, return_tensors="pt", max_length=128)
    return my_tokenizer

# Apply the tokenizer function to each row of data in the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/103 [00:00<?, ? examples/s]

Now if we take a look at our tokenized dataset, we should see each row is a list of input ids for the tokens, plus an attention mask, as expected:

In [ ]:
for i in range(0,5):
  print(tokenized_dataset['train'][i])

{'text': 'The very Republic is threatened, if involved the Sith are.', 'input_ids': [464, 845, 2066, 318, 8556, 11, 611, 2950, 262, 26455, 389, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1,

#### Training (Fine-tuning) the Model

Now we can proceed to what we really want to do - fine-tuning the model! First we need to set up a `Trainer` object from Hugging Face, as well as a `TrainingArguments` object. This was being done for us previously in the training script, where each argument we gave to the script was passed along into the Trainer.

We'll also import the `evaluate` package and load the accuracy metric from it, which will be used to evaluate the performance of our model as it is tuned:

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate

# Set up the training arguments
training_args = TrainingArguments(
    output_dir="yoda-gpt2",
    num_train_epochs=10,
    remove_unused_columns=True,
    evaluation_strategy="epoch")

# Set up the metric used to evaluate the training
metric = evaluate.load("accuracy")

Here, we need a utility function, called `compute_metrics`, to get the output probabilities (logits) for each token and token labels, then compute the predictions of the most likely token, and finally calculate the accuracy based upon these predictions with respect to the training data:

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

We'll also need to create a [DataCollator](https://huggingface.co/docs/transformers/main_classes/data_collator) object. What does the data collator do? The data collator takes the input data and creates batches of it to pass into the model. Remember, underneath it all, a large language model is still just a deep learning model, and expects batches of input data for training.

We create the data collator by importing the class from the `transformers` library, then instantiating it and passing the tokenizer object. We also set the argument `mlm=False` here, since we are doing causal language modeling, not masked language modeling:

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Ok, let's take a look at the collator in action. We can pass it a sample of data from our dataset and take a look at the output:

In [ ]:
# Sample text
texts = dataset['train'][0:4]['text']
print("Text:")
print(texts)
print("\n")

# Tokenize
print("Tokenized text:")
tokens = [tokenizer(t) for t in texts]
print(tokens)
print("\n")

# Collate
print("Collated data")

dataloader = torch.utils.data.DataLoader(dataset=tokens, collate_fn=data_collator, batch_size=4)

for batch in dataloader:
    print(batch)
print("\n")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Text:
['The very Republic is threatened, if involved the Sith are.', 'Hard to see, the dark side is. Discover who this assassin is, we must.', 'With this Naboo queen you must stay, Qui-Gon. Protect her.', 'May the Force be with you.']


Tokenized text:
[{'input_ids': [464, 845, 2066, 318, 8556, 11, 611, 2950, 262, 26455, 389, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, {'input_ids': [17309, 284, 766, 11, 262, 3223, 1735, 318, 13, 29704, 508, 428, 31120, 318, 11, 356, 1276, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, {'input_ids': [3152, 428, 36099, 2238, 16599, 345, 1276, 2652, 11, 2264, 72, 12, 38, 261, 13, 21916, 607, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, {'input_ids': [6747, 262, 5221, 307, 351, 345, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}]


Collated data
{'input_ids': tensor([[  464,   845,  2066,   318,  8556,    11,   611,  2950,   262, 26455,
           389,    13, 50256, 50256

Here we can see that the collator has reformatted the sample data (4 records) into a batch, where each key has an array of the different inputs (`input_ids`, `attention_mask`, and `labels`). We probably don't need to worry about this level of detail, but this is the format the model expects the data in, so we are really just using the data collator to restructure everything.

Finally, we can instantiate a `Trainer` object, passing in the base model to be fine-tuned, the training arguments, datasets for training and evaluation, associated evaluation metric(s), and the data collator, as defined above:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
)

Now that everything is good to go, we can simply call `trainer.train()` and Hugging Face takes care of the rest!

In [ ]:
trainer.train()

TrainOutput(global_step=130, training_loss=2.658648681640625, metrics={'train_runtime': 48.224, 'train_samples_per_second': 21.359, 'train_steps_per_second': 2.696, 'total_flos': 67282698240000.0, 'train_loss': 2.658648681640625, 'epoch': 10.0})

Great! We can visualize the model training (loss), as it is stored in the `Trainer` state:

In [ ]:
import matplotlib.pyplot as plt

# Make a dataframe of the log history
metrics_df = pd.DataFrame(trainer.state.log_history)

# Plot
metrics_df['eval_loss'].plot(marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss for GPT-2 Yoda model')
plt.show()

Finally, we should save our model from memory locally to disk. Later, we'll push the model to the Hugging Face hub:

In [ ]:
trainer.save_model()

Now, if we take a look at the folder, we can see the (updated) model weights have been saved locally, as well as the different types of model configuration files that Hugging Face expects. Here, since we set the output directory to be `yoda-distilgpt2` in the `TrainingArguments`, we can find the saved model and files there:

In [ ]:
!ls yoda-gpt2

config.json  generation_config.json  model.safetensors	runs  training_args.bin


In [ ]:
# How big is our model?
!du -h yoda-gpt2/model.safetensors

475M	yoda-gpt2/model.safetensors


### Testing our fine-tuned model
Now that we've done some (very) quick fine-tuning, let's check out the results by reloading the model and generating some text as we covered earlier. Hopefully our tuned GPT-2 model will have taken on some of the qualities of how a Jedi master speaks!

First, we can do this in the most straightforward way using a pipeline:

In [ ]:
from transformers import pipeline
from IPython.display import Markdown

# Input
input_text = "Luke, you must know that"

# Load the model into a pipeline, here the argument to is the local path, not the model name on Hugging Face hub
yodagpt2 = pipeline('text-generation',model='./yoda-gpt2', tokenizer=tokenizer, max_length=30)

# Generate the outputs
outputs = yodagpt2(input_text, num_return_sequences=4, do_sample=True, temperature=0.8, top_k=5, top_p=0.9)

# Print
for output in outputs:
  display(Markdown(output['generated_text']))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Luke, you must know that boy, and he must learn to respect you. For the boy you have become, this change is a profound one.

Luke, you must know that I have. The boy you trained, he has become. Too young to become a Jedi, too young to become an

Luke, you must know that Master Kenobi will not. Fear is the path to power. Fear leads to anger. Anger leads to hate. Anger leads

Luke, you must know that you must. I do not. Forgive my exile, you have. May the Force be with you. May the

That's looking pretty good! But we see a lot of repetition here. Let's try using the tokenizer and model directly, to apply some of the different decoding strategies we learned earlier for more varied outputs:

In [ ]:
# You can either reload the model or use it directly from the trainer object
model = trainer.model

# Input
input_text = "Luke, you must know that"

# Generate the model inputs (token ids)
model_inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Create the outputs with sampling
sample_outputs = model.generate(
    **model_inputs,
    max_new_tokens=30,
    do_sample=True,
    temperature=0.8,
    no_repeat_ngram_size=5,
    top_p=0.9,
    top_k=5,
    num_return_sequences=4,
)

# Beam sampling
beam_outputs = model.generate(
    **model_inputs,
    max_new_tokens=30,
    num_beams=5,
    no_repeat_ngram_size=5,
    early_stopping=True,
    num_return_sequences=4,
)

# Iterate over outputs, decode with tokenizer and print
display(Markdown("Sampling"))
display(Markdown("---"))
for output in sample_outputs:
  display(Markdown(tokenizer.decode(output, skip_special_tokens=True)))

display(Markdown("Beam Search"))
display(Markdown("---"))
for output in beam_outputs:
  display(Markdown(tokenizer.decode(output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sampling

---

Luke, you must know that boy. That boy...he is. May he rest in peace. May he rest. May all who love him rest.     

Luke, you must know that the Force is with you. The boy you trained, he is. May the Force be with you. May the Force rest in you. May the

...

---

Luke, you must know that the Force is with you. May the Force be with you. Your apprentice, Qui-Gon Jinn. May all that is left of you

Luke, you must know that the Force is with you. May the Force be with you. Your apprentice, Qui-Gon Jinn. May peace and security rest in your

Luke, you must know that the Force is with you. May the Force be with you. Your apprentice, Qui-Gon Jinn. May peace and harmony rest in your

Luke, you must know that the Force is with you. May the Force be with you. Your apprentice, Qui-Gon Jinn. May the Force rest in your power

### Applying what we've learned: Build your own "Bard"

Use what you've learned above to fine-tuning GPT-2 on the [tiny shakespeare](https://github.com/nlpfromscratch/datasets/tree/master/tinyshakespeare) dataset.

You'll need to apply the 4 steps we've seen above:
1. Load the dataset and tokenize it
2. Load the base model and tokenizer
3. Set up your evaluation metric, data collator, and training arguments
4. Train the model and save it

<center>
<img src="https://drive.google.com/uc?export=download&id=1HdtuTtpiXm_oUHyf7IB4jMZ6hSVKT5v0" width="150px"/>
</center>

This dataset is located in `datasets/tinyshakespeare`. Fine-tune the model for a single epoch (this should take about ~8-9 mins for training), then generate some outputs from your model, either using a pipeline or the tokenizer and model directly, to see the results.

In [ ]:
!ls datasets/tinyshakespeare

input.txt  README.md


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM, Trainer, DataCollatorForLanguageModeling, pipeline
import evaluate

# 1. Load base model and tokenizer

# Use GPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Instantiate model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

In [ ]:
# 2. Load dataset and tokenize
data_files = {"train": "input.txt" }
dataset_name = '/content/datasets/tinyshakespeare/'
dataset = load_dataset(dataset_name, data_files=data_files)

def tokenize_function(data):
    my_tokenizer = tokenizer(data["text"], padding=True, truncation=True, return_tensors="pt", max_length=128)
    return my_tokenizer

tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [ ]:
# 3. Set up metric, training arguments, and data collator
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="tinyshakespeare-gpt2",
    evaluation_strategy="epoch",
    remove_unused_columns=True,
    num_train_epochs=1)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# 4. Train & save model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
)

trainer.train()

trainer.save_model()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
# 4. Generate text
input_text = "In fair Verona"

generator = pipeline('text-generation',model='./tinyshakespeare-gpt2', tokenizer=tokenizer, max_length=100)

outputs = generator(input_text, num_return_sequences=4)

for output in outputs:
  display(Markdown(output['generated_text']))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In fair Verona have many words, and I'll tell thee in what. For you will hear me speak it. What, Camillo? look you, my king! the king is gone. O, my peace! thou meanest it; the Duke of Buckingham is gone! O, my king! thou meanest it, my peace! look you, my king! the Duke of Gloucester is gone! O, my king! I swear! it is death! Come, my lord

In fair Verona; how I am to know what? I cannot tell; but you know: I have been in London, my man: what, sir? sir? what, sir? sir? how, sir? sir? what, sir? sir! you have been there? you know no man: you know he cannot say so. What, sir? sir? sir, sir, sir! sir, I am a fool. Do you speak, sir? sir, sir, sir

In fair Verona, on Thursday's Wednesday. What the devil's you, sir? I speak ill of him at his company. What was it, sir? how's he, sir? speak ill of him: if he have come to't, say true 'tis, be very; for the day. I know you come. Did he come? Did he go, sir? if you call me so? were he not, 'Tis you the duke! who is he?

In fair Verona, I was a man's son, and I would have my lord to die; but he'll keep, for he's not worthy. Come, marry us soon. What name shall we play, boys? Claudio? Claudio, Claudio, Claudio, come! Claudio, Claudio, son! Claudio! Claudio, son! Claudio! Claudio, son! Claudio, son! Claudio, son! Claudio! Claudio,

### Saving our Model to the Hugging Face Hub
Now that we have created a fine-tuned model, we'd like to push our model to the Hub, such that others can use it, or we can easily load it from the cloud.

To do this, we need to login to our Hugging Face account, then create a new token with write access, as we will be pushing models to the hub:

<img src="https://drive.google.com/uc?export=download&id=16BT1moRWdRzayowW6vKw9-WhOECjry1t"/>

Copy the next token to your clipboard. Now we can log in to the Hugging Face inside of the notebook using the newly created token and the code below:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Paste your token in the box when you run the cell above, uncheck the box that says "Save as git credential" and hit ok. You should log in successfully. Now we are ready to push our model to the hub! We can do this using the `trainer` we set up before:

In [ ]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1700494722.a0ef25d8bf46.3659.1:   0%|          | 0.00/6.65k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

'https://huggingface.co/nlpfromscratch/tinyshakespeare-gpt2/tree/main/'

We should also push our tokenizer to the same repo, in order to allow the model and tokenizer to be loaded via a pipeline:

In [ ]:
# Get the model id
hub_model_id = trainer.hub_model_id
print(hub_model_id)

# Add the tokenizer
tokenizer.push_to_hub(hub_model_id)

nlpfromscratch/tinyshakespeare-gpt2


CommitInfo(commit_url='https://huggingface.co/nlpfromscratch/tinyshakespeare-gpt2/commit/ec5cb90ea08fef2ea687f8ea5d0bd8acf49ebaad', commit_message='Upload tokenizer', commit_description='', oid='ec5cb90ea08fef2ea687f8ea5d0bd8acf49ebaad', pr_url=None, pr_revision=None, pr_num=None)

Great! Now if you reload your Hugging Face page, you should see that the model has been successfully uploaded. Now anyone can download your model and use it:

In [ ]:
from transformers import pipeline
from IPython.display import Markdown

# Input
input_text = "In fair Verona"

# Model name
your_hf_username = "nlpfromscratch" # PUT YOUR username here
model_name = f"{your_hf_username}/tinyshakespeare-gpt2"

# Load the model into a pipeline, here the argument to is the local path, not the model name on Hugging Face hub
yodagpt2 = pipeline('text-generation', model=model_name, max_length=30)

# Generate the outputs
outputs = yodagpt2(input_text, num_return_sequences=4, do_sample=True, temperature=0.8, top_k=5, top_p=0.9)

# Print
for output in outputs:
  display(Markdown(output['generated_text']))

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In fair Verona. Come, I'll make you a man: I'll make you a man: I'll make you a man. Come,

In fair Verona. What's the matter? I am not to speak. Farewell, my lord. Farewell. Farewell, my lord

In fair Verona, to be revenged. I will not stay: I will go. I will not stay: my lord, my lord!

In fair Verona, I will go to him. Come, come; I will be gone. I'll be gone. What, ho! what

Amazing! Now we have uploaded our fine-tuned model to the Hugging Face hub along with its tokenizer, and anyone else can easily download it and use it in a pipeline!

As we'll be moving into the next section, please restart the Colab runtime here to clear RAM and run the cell below:

In [ ]:
import torch
from IPython.display import Markdown
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

### Model Quantization and Parameter Efficient Fine-tuning (PEFT)

In this section we will explore two concepts for making working with LLMs more tractable without extensive compute or memory requirements. *Model quantization* and *parameter efficient fine-tuning*.

While the former can be used just to speed up model predictions, or *inference*, it is often used in conjunction with the latter for reducing the computational requirements for training very large models.

#### Model Quantization

 Quantization is a model size reduction technique that loads the model weights in a lower precision than they were originally trained in, resulting in a reduction in the storage and compute costs for the model.

 This can mean taking a model which originally had its weights stored in a high-precision format, such as 32 or 16-bit floating point, and converting them into a lower precision such as 8-bit (or even 4-bit!) integer values.

 Surprisingly, even with this loss of precision, many sophisticated LLMs are still found to perform quite well after quantization.

 Model quantization approaches do not just simply round or truncate the weight values, they are converted using a formula like the below in the [affine quantization scheme](https://pytorch.org/blog/quantization-in-practice/#affine-and-symmetric-quantization-schemes):

 $x_q = round(x/S + Z)$

 where:
 - $x_q$ is the quantized value
 - $x$ is the original value
 - $S$ is the *scale factor*
 - $Z$ is the zero point

Optimal values can all be determined as part of the quantization process in calibration.

There are also well-known more sophisticated approaches for quantization such as [GPT-Q](https://arxiv.org/abs/2210.17323) and [Activation-aware Weight Quantization (AWQ)](https://github.com/mit-han-lab/llm-awq) which have been applied to create qunatized versions of popular models which are highly optimized inference. In the Hugging Face space, Tom Jobbins (*a.k.a* [The Bloke](https://huggingface.co/TheBloke)) is a well-known individual for producing many quantized versions and variations of popular models.

Let's try loading a large version of GPT-2 using qunatization, and then do some inference. Here we will use [GPT2-XL](https://huggingface.co/gpt2-xl), the 1.5B parameter version of GPT-2, which weighs in at ~6.5GB! All we need to do is pass the `load_in_4bit=True` parameter when loading the model to apply quantization:

In [ ]:
model_id = "gpt2-xl"

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
f"{model.num_parameters():,}"

'1,557,611,200'

We can see that each linear layer in the model has been replaced with a 4-bit layer:

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-47): 48 x GPT2Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear4bit(in_features=1600, out_features=4800, bias=True)
          (c_proj): Linear4bit(in_features=1600, out_features=1600, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=1600, out_features=6400, bias=True)
          (c_proj): Linear4bit(in_features=6400, out_features=1600, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1600,), eps=1e-05, element

Now, let's generate some text. We know how to do this now, so leverage some of our earlier code from before, using temperature and top-p sampling:

In [ ]:
# Generate the model inputs (token ids)
input_text = "The rain in Spain"

model_inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Create the outputs with sampling
sample_outputs = model.generate(
    **model_inputs,
    max_new_tokens=30,
    do_sample=True,
    temperature=0.8,
    no_repeat_ngram_size=5,
    top_p=0.9,
    top_k=5,
    num_return_sequences=3,
)

# Iterate over outputs, decode with tokenizer and print
for output in sample_outputs:
  display(Markdown(tokenizer.decode(output, skip_special_tokens=True)))
  display(Markdown("---"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


The rain in Spain was so heavy that the wind was so fierce that it was almost as if it was a thunderstorm. The wind was so fierce, in fact,

---

The rain in Spain was a little more than usual, but it was not the worst I've ever experienced. The weather was perfect. I was on my way to a

---

The rain in Spain is the most intense in the world. It has been raining for a long time. But the rain has been getting more intense. The rain has been

---

Great! We can now efficiently perform inference using very large LLMS via quantization 👍 You can read more about loading quantized models in the Hugging Face documentation here: [Quantize 🤗 Transformers models](https://huggingface.co/docs/transformers/main_classes/quantization)

#### Fine-tuning a quantized LLM using PEFT

Now that we have sucessfully loaded a very large transformer model, let's move on to seeing how we can fine tune it using a parameter efficient fine-tuning, or [PEFT](https://github.com/huggingface/peft), approach.

Luckily for us this is easy to do in Hugging Face using the `peft` library which we installed at the beginning of this notebook.

Here we will fine-tune a quantized version of GPT2-XL using [Low Rank Adaption (LoRA)](https://github.com/microsoft/LoRA). This approach was first introduced by researchers from Microsoft in 2021.

Instead of updating all the weights in a model, LoRA is a reparameterization method that introduces a smaller number new weights via two matrices which decompose the weights into a lower rank representation, and only these weights are updated during the training process.<br/><br/>

<center>
<img src="https://drive.google.com/uc?export=download&id=1H4KTXgCWHRwSCk1uxelCDsQCfziJCogl" width="50%"/>
</center>
<center>
<caption> Diagram of LoRA, from the original paper </caption>
</center>

Luckily, this is all easy enough to do as it is implemented in Hugging Face in the `peft` library.

First, we need to prepare the model for quantized training:

In [ ]:
from peft import prepare_model_for_kbit_training

# Reload the quantized model and tokenizer
#model = AutoModelForCausalLM.from_pretrained("gpt2-xl", load_in_4bit=True)
#tokenizer = AutoTokenizer.from_pretrained("gpt2-xl")
tokenizer.pad_token = tokenizer.eos_token

# Enable ch
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

Then, we import a [LoRA configuration](https://huggingface.co/docs/peft/conceptual_guides/lora#common-lora-parameters-in-peft), and then apply it to the model to use LoRA in fine-tuning:

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config).to(device)

Using this utility function, we can take a look at the number of parameters in the model which will be updated as part of the fine-tuning:

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model)

trainable params: 2,457,600 || all params: 822,788,800 || trainable%: 0.2986914746530337


We can see that total number of parameters counted here has been greatly reduced, this is due to the model weights being loaded in 4-bit as part of quantization. The total number of trainable parameters with LoRA here is still ~2.5M.

Now that the model is set up with LoRA, the remaining steps are the same as for regular fine-tuning! We import the data, set up a training configuration and `Trainer` object, and the start the fine-tuning:

In [ ]:
from datasets import load_dataset
# 1. Load dataset and tokenize
data_files = {"train": "yoda.csv"}
dataset_name = 'datasets/yoda/'
dataset = load_dataset(dataset_name, data_files=data_files)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(data):
    my_tokenizer = tokenizer(data["text"], padding=True, truncation=True, return_tensors="pt", max_length=128)
    return my_tokenizer

tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Map:   0%|          | 0/103 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Set up the training arguments
training_args = TrainingArguments(
        output_dir="yoda-gpt2xl-lora",
        num_train_epochs=10,
        fp16=True,
        optim="paged_adamw_8bit"
    )

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Trainer
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    args=training_args,
    data_collator=data_collator,
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=130, training_loss=3.9429307204026443, metrics={'train_runtime': 91.3213, 'train_samples_per_second': 11.279, 'train_steps_per_second': 1.424, 'total_flos': 429305456832000.0, 'train_loss': 3.9429307204026443, 'epoch': 10.0})

Now let's test it out:

In [ ]:
input_text = "Luke, you must know that "

model_inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Create the outputs with sampling
outputs = trainer.model.generate(
    **model_inputs,
    max_new_tokens=30,
    do_sample=True,
    no_repeat_ngram_size=2,
    temperature=0.8,
    top_p=0.95,
    top_k=5,
    num_return_sequences=4,
)

# Iterate over outputs, decode with tokenizer and print
display(Markdown("---"))
for output in outputs:
  display(Markdown(tokenizer.decode(output, skip_special_tokens=True)))
  display(Markdown("---"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---

Luke, you must know that  it is not the will of the Force that the Jedi should be chosen to lead us. ________ ______________________________ __________________ ________________________________________

---

Luke, you must know that  Luke. I am the only one who knows about him. He is a Jedi, Luke. The only Jedi left. Luke, I need you

---

Luke, you must know that ichthar's not a Jedi. It's a Sith. The Dark side of the Force. I am the Dark Lord of Darth Vader. Your

---

Luke, you must know that  the Jedi are a                                       ...          

---

Great! With just a few epochs of training, we seem to have significantly altered the behavior of the model as before, only with tuning a much smaller subset of parameters, making it feasible with the computing resources we have available.

To eliminate latency in inference and use the model as a standalone model without LoRA, we can [merge the adapter weights back into the base model](https://huggingface.co/docs/peft/conceptual_guides/lora#merge-lora-weights-into-the-base-model). This should be done before pushing the model to the Hub if you wish to use the model as a standalone model.

<center>
<img src="https://drive.google.com/uc?export=download&id=1H_FgH-_axdQohpDe5cOhQtkh1DUrk3bU">
</center>
<center>
<caption> Merging the adapter weights </caption>
</center>

In [ ]:
# Model is a PEFT model
model.__class__

peft.peft_model.PeftModelForCausalLM

In [ ]:
# Merge the LoRA weights back into the model
merged_model = model.merge_and_unload()

In [ ]:
# Merged model is of same type as base model
merged_model.__class__

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

### Further optimizing LoRA: QLoRA

Building on the work of the research of the team at Microsoft, researchers from University of Washington developed [QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314) in May of 2023.

[QLoRA](https://github.com/artidoro/qlora) makes parameter efficient fine-tuning even more so by using 4-bit quantization for the model to be tuned, introducing a new data type called 4-bit NormalFloat (NF4), and using "double quantization", where the quantization parameters are also quantized.

This can be accomplished in code by using a `bitsandbytes` config and with the following parameters:

```python
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

...
model = AutoModelForCausalLM.from_pretrained(
  "gpt2-xl",
  quantization_config=bnb_config
  )

```

You can see an example of using QLoRA in Hugging Face [in this example notebook](https://colab.research.google.com/drive/1VoYNfYDKcKRQRor98Zbf2-9VQTtGJ24k?usp=sharing) and more details in the [official blog post](https://huggingface.co/blog/4bit-transformers-bitsandbytes) from Hugging Face.

A notable output of the QLoRA research was that of the [Guanco model](https://huggingface.co/spaces/uwnlp/guanaco-playground-tgi) family which was fine-tuned on LLaMA 2.

### Homework: Fine-tuning a quantized generative text model

Apply what you've learned about fine-tuning and model quantization and use LoRA to fine-tune GPT-2XL on the dialog from Amazon's TV series [The Peripheral](https://github.com/nlpfromscratch/datasets/tree/master/peripheral).

<center>
<img src="https://drive.google.com/uc?export=download&id=1HeLMEpCcP9vsfgSdx8upW0TyC--zbvEn" width="50%"/>
</center>

After you've done some fine-tuning, see if you can recover character names from the show with prompting (*e.g.* Flynn Fisher, Wilf Netherton, Corbell Pickett).

In [ ]:
!ls datasets/peripheral

## Conclusion 🏁

We have covered a lot of ground in both working with generative large language models for inference, as well as methods for efficiently fine-tuning them with limited computational resources. However, we have really just scratched the surface of LLMs for text, and there is much more that makes up the incredibly sophisticated models which represent the state of the art.

- **Training "chat" models**: By changing the [format of the input](https://huggingface.co/docs/transformers/main/en/chat_templating) and prediction task, and applying more sophisticated approaches such as [instruction tuning](https://arxiv.org/abs/2308.10792).
- **Reinforcement learning from human feedback (RLHF)**: Now a standard part of LLM development but is non-trivial for the individual. However, this is possible to do with a base model and the right datasets using the [trl library](https://huggingface.co/docs/trl/index).
- **Retrieval Augmented Generation**: Or [RAG](https://proceedings.neurips.cc/paper/2020/hash/6b493230205f780e1bc26945df7481e5-Abstract.html), for combining the generative capabilities of LLMs with search and retrieval from a datastore, to reduce "hallucinations" and be able to have a language model work against a knowledge base.

LLMs represent the state of the art in generative text models and have rapidly transformed this space (and many other domains) in a very short order. Knowing how how they function, how to use them, and being aware of their shortcomings allows them to be judiciously applied to the right use cases to unlock business value and build product of value to the end-user.

### References and Further Reading

- Vaswani, Ashish, et al. (1 Aug. 2023). *Attention Is All You Need*. http://arxiv.org/abs/1706.03762.

- *How to generate text: using different decoding methods for language generation with Transformers* (2020, March 1). Patrick von Platen (Hugging Face). Retrieved November 2023. https://huggingface.co/blog/how-to-generate

- *Fine-tune a pretrained model* (n.d.) Hugging Face. Retrieved November 2023. https://huggingface.co/docs/transformers/training#fine-tune-a-pretrained-model

- *Quantize Transformers Models* (n.d.) Hugging Face. Retrieved November 2023. https://huggingface.co/docs/transformers/main_classes/quantization

- *LoRA: Low-Rank Adaptation of Large Language Models* (n.d.) Microsoft. Retrieved November 2023. https://github.com/microsoft/LoRA

- Dettmers, Tim; Pagnoni, Artidoro; and Holtzman, Ari. (May 2023). *QLoRA: Efficient Finetuning of Quantized LLMs*. https://arxiv.org/abs/2305.14314

- *Practical Tips for Finetuning LLMs Using LoRA (Low-Rank Adaptation)* (November 19th, 2023). Raschka, Sebastian. https://magazine.sebastianraschka.com/p/practical-tips-for-finetuning-llms



----

<table border="0" bgcolor="white">
  <tr></tr>
  <tr>
      <th align="left" style="align:left; vertical-align: bottom;"><p>Copyright NLP from scratch, 2023.</p></th>
      <th aligh="right" width="33%"><a href="https://www.nlpfromscratch.com?utm_source=notebook&utm_medium=nb-footer-img"><img src="https://drive.google.com/uc?export=view&id=1-lt6Uft8lgBG9jPD0dO6w3dAcv_EUQRP"></th>
</tr>
</table>